In [2]:
import pandas as pd
import requests

This notebook gathers data from the census API 

In [22]:
#got state number list from this github https://gist.github.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53
census_state_list_url = "https://gist.githubusercontent.com/dantonnoriega/bf1acd2290e15b91e6710b6fd3be0a53/raw/11d15233327c8080c9646c7e1f23052659db251d/us-state-ansi-fips.csv"

In [31]:
#need converter to keep leading zero
states = pd.read_csv(census_state_list_url,converters={' st': lambda x: str(x)})
states_list = states[" st"]

In [112]:
def get_data(year,acs,state,codes):
    """
    Load data from API. modify the url to change the items being requested. Also if adding new items to call, add them to the "codes" dictionary above
    so they they get proper column labels. 
    inputs:
        year (str) year of the call 
        acs(str) 1 or 5 year acs 
        state (str) which state - should run through state list - of form 01-50.
        codes (str) codes for data you want to load 
        
    """
    #use the URL below for census tract level 
    #url = "https://api.census.gov/data/"+year+"/acs/acs"+acs+"?get=NAME,"+codes +"&for=tract:*&in=state:"+state+"&in=county:*"
    
    #the URL below is for county level 
    url = "https://api.census.gov/data/"+year+"/acs/acs"+acs+"?get=NAME,"+codes +"&for=county:*&in=state:*" 
    #+state

    #print(url)
    response = requests.get(url)
    data_1 = pd.DataFrame(response.json())

    return data_1 


In [61]:
def make_header(df,codes):
	"""
	do some basic cleanup of the header row 
	"""
	new_header = df.iloc[0] #grab the first row for the header
	df = df[1:] #take the data less the header row
	df.columns = new_header
	df.columns = [codes[x] for x in df.columns]
	return df 

In [52]:
#to make column names need to make dictionary of codes to names acs_5
codes = dict(zip(acs_5.code, acs_5.label))
tract_dict = {"NAME":"Name","state":"State","county":"County","tract":"Tract"}
codes.update(tract_dict) 

In [101]:
#to make column names need to make dictionary of codes to names acs_1_2018
codes_1_2018 = dict(zip(acs_1_2018.code, acs_1_2018.label))
tract_dict = {"NAME":"Name","state":"State","county":"County"}
codes_1_2018.update(tract_dict) 

In [100]:
#to make column names need to make dictionary of codes to names acs_1_2014
codes_1_2014 = dict(zip(acs_1_2014.code, acs_1_2014.label))
tract_dict = {"NAME":"Name","state":"State","county":"County"}
codes_1_2014.update(tract_dict) 

In [5]:
#read in list of desired codes for variables 
df_key = pd.read_excel("ACS data dl.xlsx")

In [8]:
df_key.survey.unique()

array(['ACS 5 year 2018', 'ACS 1 year 2018', 'ACS 1 year 2014'],
      dtype=object)

In [11]:
#subset the data based on calls to different datasets 
acs_5 = df_key[df_key.survey == 'ACS 5 year 2018']
acs_1_2018 = df_key[df_key.survey == 'ACS 1 year 2018']
acs_1_2014 = df_key[df_key.survey == 'ACS 1 year 2014']

In [87]:
#make string of codes we want 
def make_string(df):
    string = ""
    for i in list(df["code"].unique()):
        i = i + ","
        string += i 
    string = string[:-1]
    return string

In [ ]:
#make different variable name keys for each substring 
string = make_string(acs_5)
string_acs_1_2014 = make_string(acs_1_2014)
string_acs_1_2018 = make_string(acs_1_2018)

In [106]:
def run_the_states(states_list,codes,year,acs,string):
    """
    states_list (list) list of strings of state codes (01, etc)
    codes (str) single string of all variable codes 
    year (str) year of data
    acs (str) 1 or 5 year acs data 
    codes (dict) dictionary mapping of ACS variable name codes to real names 
        Run through list of states to get acs data from each census tract. 
    
    """

    final_df = pd.DataFrame()
    df = []
    for i in states_list:
        try:
            new_state = get_data(year ,acs ,i , string)
        except Exception as e:
            print(i," doesn't work", e)
        try:
            new_state = make_header(new_state, codes)
        except Exception as e:
            print(i," col name doesn't work", e)
        
        try:
            df.append(new_state)
        
        except Exception as e:
            print("cant add new info",file_name, e)
        
    final_df = final_df.append(df, True)   
    return final_df

In [111]:
def run_county_level(states_list,codes,year,acs,string):
    try:
        final_df = get_data(year ,acs ,i , string)
    except Exception as e:
        print(i," doesn't work", e)
    try:
        final_df = make_header(final_df, codes)
    except Exception as e:
        print(i," col name doesn't work", e)

    return final_df

In [72]:
#gather all acs5 data.
full_df = run_the_states(states_list,codes)

https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E&for=tract:*&in=state: 01&in=county:*
https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E&for=tract:*&in=state: 02&in=county:*
https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B2302

https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E&for=tract:*&in=state: 28&in=county:*
https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E&for=tract:*&in=state: 29&in=county:*
https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B2302

https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E&for=tract:*&in=state: 54&in=county:*
https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B23025_006E,B23025_007E&for=tract:*&in=state: 55&in=county:*
https://api.census.gov/data/2018/acs/acs5?get=NAME,B17020_001E,B17020_002E,B17020_010E,B03002_001E,B03002_004E,B03002_003E,B25077_001E,B25031_001E,B25031_002E,B25031_003E,B25031_004E,B25031_005E,B25031_006E,B25031_007E,B25064_001E,B23025_001E,B23025_002E,B23025_003E,B23025_004E,B23025_005E,B2302

In [113]:
#gather all AC1 2018 data 
acs_1_2018_full_data = run_county_level(states_list,codes_1_2018,"2018","1",string_acs_1_2018)

In [119]:
#gather all AC1 2014 data 
acs_1_2014_full_data = run_county_level(states_list,codes_1_2014,"2014","1",string_acs_1_2014)

In [116]:
#each data set had 838 unique counties, including PR
len(acs_1_2018_full_data["Name"].unique())

838

In [117]:
len(acs_1_2018_full_data["State"].unique())

52

In [74]:
#write ACS5 data to csv 
full_df.to_csv("ACS5Year_Tract_Level_Poverty.csv")

In [118]:
#write ACS_1_18 data to csv 
acs_1_2018_full_data.to_csv("ACS1_2018_County_Level_Poverty.csv")

In [121]:
#write ACS_1_14 data to csv 
acs_1_2014_full_data.to_csv("ACS1_2014_County_Level_Poverty.csv")

In [122]:
acs_1_2014_full_data

,Name,Estimate!!Total,Estimate!!Total!!In labor force,Estimate!!Total!!In labor force!!Civilian labor force,Estimate!!Total!!In labor force!!Civilian labor force!!Employed,Estimate!!Total!!In labor force!!Civilian labor force!!Unemployed,Estimate!!Total!!In labor force!!Armed Forces,Estimate!!Total!!Not in labor force,Estimate!!Median household income in the past 12 months (in 2014 inflation-adjusted dollars),State,County
1,"Baldwin County, Alabama",160204,94387,94063,87303,6760,324,65817,48461,01,003
2,"Calhoun County, Alabama",93147,53824,53709,48114,5595,115,39323,41428,01,015
3,"Cullman County, Alabama",64854,32792,32584,30722,1862,208,32062,38261,01,043
4,"DeKalb County, Alabama",55547,28357,28357,27002,1355,0,27190,35023,01,049
5,"Elmore County, Alabama",64514,35680,34980,31607,3373,700,28834,55530,01,051
...,...,...,...,...,...,...,...,...,...,...,...
824,"Ponce Municipio, Puerto Rico",123930,47264,47136,38976,8160,128,76666,15935,72,113
825,"San Juan Municipio, Puerto Rico",302536,154849,154818,131229,23589,31,147687,20234,72,127
826,"Toa Alta Municipio, Puerto Rico",59563,28829,28829,26279,2550,0,30734,29341,72,135
827,"Toa Baja Municipio, Puerto Rico",68043,38536,38536,30911,7625,0,29507,22166,72,137
